In [1]:
import pandas as pd

In [19]:
test = pd.read_json('compositions.json')

In [20]:
string = test.journeySections

In [53]:
json_dict = test.journeySections.to_dict()

In [30]:
test

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainCategory,trainType,version,journeySections
0,464,2023-08-25,10,vr,Long-distance,IC,286330743404,[{'beginTimeTableRow': {'stationShortCode': 'P...


In [54]:
t

{0: [{'beginTimeTableRow': {'stationShortCode': 'PRI',
    'stationUICCode': 220,
    'countryCode': 'FI',
    'type': 'DEPARTURE',
    'scheduledTime': '2023-08-25T07:03:00.000Z'},
   'endTimeTableRow': {'stationShortCode': 'TPE',
    'stationUICCode': 160,
    'countryCode': 'FI',
    'type': 'ARRIVAL',
    'scheduledTime': '2023-08-25T08:36:00.000Z'},
   'locomotives': [{'location': 4,
     'locomotiveType': 'Sr2',
     'powerType': 'Electric'}],
   'wagons': [{'wagonType': 'Edo',
     'location': 1,
     'salesNumber': 1,
     'length': 2740,
     'pet': True},
    {'wagonType': 'CEd', 'location': 2, 'salesNumber': 2, 'length': 2640},
    {'wagonType': 'Edfs',
     'location': 3,
     'salesNumber': 4,
     'length': 2640,
     'playground': True,
     'disabled': True}],
   'totalLength': 99,
   'maximumSpeed': 140,
   'attapId': 365205565,
   'saapAttapId': 365207015}]}

In [48]:
json_dict

{'trainNumber': {0: 464},
 'departureDate': {0: '2023-08-25'},
 'operatorUICCode': {0: 10},
 'operatorShortCode': {0: 'vr'},
 'trainCategory': {0: 'Long-distance'},
 'trainType': {0: 'IC'},
 'version': {0: 286330743404},
 'journeySections': {0: [{'beginTimeTableRow': {'stationShortCode': 'PRI',
     'stationUICCode': 220,
     'countryCode': 'FI',
     'type': 'DEPARTURE',
     'scheduledTime': '2023-08-25T07:03:00.000Z'},
    'endTimeTableRow': {'stationShortCode': 'TPE',
     'stationUICCode': 160,
     'countryCode': 'FI',
     'type': 'ARRIVAL',
     'scheduledTime': '2023-08-25T08:36:00.000Z'},
    'locomotives': [{'location': 4,
      'locomotiveType': 'Sr2',
      'powerType': 'Electric'}],
    'wagons': [{'wagonType': 'Edo',
      'location': 1,
      'salesNumber': 1,
      'length': 2740,
      'pet': True},
     {'wagonType': 'CEd', 'location': 2, 'salesNumber': 2, 'length': 2640},
     {'wagonType': 'Edfs',
      'location': 3,
      'salesNumber': 4,
      'length': 2640,


In [42]:
isinstance(json_dict['journeySections'], type(dict))

True